# Evaluating the karabo detection source
In this notebook, we will evaluate how well the detection source of karabo works (check the meeting of the 13.10.22 on gitlab for more infos: here: https://gitlab.unige.ch/Patrick.Arlt/master-thesis/-/issues/12


In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import numpy as np
import math
import matplotlib.pyplot as plt

import myUtils

from astropy.utils.data import get_pkg_data_filename
from astropy.io import fits
from astropy.table import Table
from astropy.coordinates import SkyCoord
import astropy.units as u


sys.path.insert(0, '/Users/patoch/Desktop/Patoch/Uni/2nd-Year/MasterThesis/master-thesis/Karabo-Pipeline')
from karabo.imaging import image
from karabo.sourcedetection import result
from tqdm import tqdm

# Getting the right fits according to the keys given

In [2]:
data_path = '/Users/patoch/Desktop/Patoch/Uni/2nd-Year/MasterThesis/master-thesis/Dataset.nosync'

In [3]:
keys = np.load(os.path.join(data_path, "Sources_detection/sky_keys.npy"))
indices = np.load(os.path.join(data_path, "Sources_detection/seed_0/test_idx.npy"))

print(len(keys))
print(len(indices))

9164
916


In [4]:
testSet_path = []   # list containing all the fits file path for the test 

for ind in indices:
    key = keys[ind]

    data = os.path.join(data_path, 'clean_gaussian', "clean_gaussians_" + key + ".fits")
    testSet_path.append(data)


In [5]:
# listDetectedSources_test = []
# %%capture
# for i in tqdm(range(len(testSet_path))):
#     path = testSet_path[i]
    
#     image_tmp = image.Image.read_from_file(path)
#     detection = result.SourceDetectionResult.detect_sources_in_image(image_tmp, quiet = True);
#     listDetectedSources_test.append(detection.get_pixel_position_of_sources())

In [6]:
saveSourceDetection_path = "/Users/patoch/Desktop/Patoch/Uni/2nd-Year/MasterThesis/master-thesis/Dataset.nosync/sourceDetection_Karabo"

In [7]:
# myUtils.saveSourceDetection(listDetectedSources_test, saveSourceDetection_path, "clean_gaussians_", indices, keys)  

In [8]:
listDetectedSources_test = myUtils.loadSourceDetection(saveSourceDetection_path, "clean_gaussians_", indices, keys)
print(len(listDetectedSources_test))
print(listDetectedSources_test[0].shape)


916
(2, 5)


# Comparision with the real Sources

### Loading the real source coordinates

In [9]:
listRealDetection_test = myUtils.loadAllCatFile("/Users/patoch/Desktop/Patoch/Uni/2nd-Year/MasterThesis/master-thesis/Dataset.nosync/Sources_detection/cat", "gaussians_", indices, keys)
print(len(listRealDetection_test))

916


## Perform the comparaision

### Bunch of usefull functions

In [10]:
# -------- Testing purposes --------
img_tmp = image.Image.read_from_file(testSet_path[0])

pixelcoord = myUtils.RaDec2pixels(listRealDetection_test[0], img_tmp.header)
print(" ---- From Ra & Dec coordinates to Pixel Coordinates ----")
print("Real observations: ", pixelcoord)
print("Karabo: ", np.round(listDetectedSources_test[0]))
print()
print(" ---- From Pixel Coordinates to Ra & Dec coordinates ----")
print("Real observations: ", listRealDetection_test[0])
print("Karabo: ", myUtils.pixels2RaDec(listDetectedSources_test[0][0][0], listDetectedSources_test[0][1][0], img_tmp.header))
print("Karabo: ", myUtils.pixels2RaDec(listDetectedSources_test[0][0][1], listDetectedSources_test[0][1][1], img_tmp.header))


 ---- From Ra & Dec coordinates to Pixel Coordinates ----
Real observations:  [[166. 206.]
 [336. 156.]]
Karabo:  [[166. 206. 204. 359. 433.]
 [336. 156. 156. 260. 316.]]

 ---- From Pixel Coordinates to Ra & Dec coordinates ----
Real observations:  [[149.06166667   3.62005556]
 [149.06055556   3.61505556]]
Karabo:  [149.06165641   3.62005319]
Karabo:  [149.06054717   3.61505995]


In [11]:
# -------- Testing purposes --------
result = myUtils.compareRealAndDetectedSources_pixels([listDetectedSources_test[0]], myUtils.RaDec2pixels([listRealDetection_test[0]], img_tmp.header))
print("Detected Sources: \n", listDetectedSources_test[0]," \n")
print("Real Sources: \n", myUtils.RaDec2pixels([listRealDetection_test[0]], img_tmp.header), " \n")
print("Result (TP, FP, FN): ", result)

Detected Sources: 
 [[166.36922983 206.30190138 203.99364085 358.61539154 432.70799906]
 [335.91483971 156.15837177 155.65601904 260.20109428 315.70767354]]  

Real Sources: 
 [[[166. 206.]
  [336. 156.]]]  

Result (TP, FP, FN):  [2 3 0]


### Do the comparaison

In [12]:
# Generate the list of real sources in pixel coordinates
listRealDetection_test_px = []
for ind in tqdm(range(len(listRealDetection_test))):
    listRealDetection_test_px.append(myUtils.RaDec2pixels(listRealDetection_test[ind], image.Image.read_from_file(testSet_path[ind]).header))


100%|██████████| 916/916 [00:15<00:00, 57.83it/s]


In [28]:
# Compute the TP, FP and FN for the whole test set
resAllComparison = myUtils.compareRealAndDetectedSources_pixels(listDetectedSources_test, listRealDetection_test_px)
print("Result (TP, FP, FN): ", resAllComparison)

Result (TP, FP, FN):  [2781 5320   55]


In [29]:
# Compute the TP, FP and FN for each file
listComparison_test= []
for ind in tqdm(range(len(listDetectedSources_test))):
    listComparison_test.append(myUtils.compareRealAndDetectedSources_pixels([listDetectedSources_test[ind]], [listRealDetection_test_px[ind]]))
listComparison_test = np.array(listComparison_test)
print("Result (TP, FP, FN): \n", listComparison_test)
print()

print("Mean TP: ", np.mean(listComparison_test[:,0]))
print("Mean FP: ", np.mean(listComparison_test[:,1]))
print("Mean FN: ", np.mean(listComparison_test[:,2]))



100%|██████████| 916/916 [00:00<00:00, 3473.63it/s]

Result (TP, FP, FN): 
 [[ 2  3  0]
 [ 3  9  0]
 [ 5  0  0]
 ...
 [ 3  1  0]
 [ 3 10  0]
 [ 5  2  0]]

Mean TP:  3.0360262008733625
Mean FP:  5.807860262008734
Mean FN:  0.060043668122270744


# Compute purity and Completenss


In [30]:
purityAll = resAllComparison[0] / (resAllComparison[0] + resAllComparison[1])
CompletnessAll = resAllComparison[0] / (resAllComparison[0] + resAllComparison[2])
print("Purity: ", purityAll)
print("Completness: ", CompletnessAll)

Purity:  0.3432909517343538
Completness:  0.9806064880112835


In [31]:
meanPurity = np.mean(listComparison_test[:,0]) / (np.mean(listComparison_test[:,0]) + np.mean(listComparison_test[:,1]))
meanCompletness = np.mean(listComparison_test[:,0]) / (np.mean(listComparison_test[:,0]) + np.mean(listComparison_test[:,2]))
print("Mean Purity: ", meanPurity)
print("Mean Completness: ", meanCompletness)


Mean Purity:  0.34329095173435376
Mean Completness:  0.9806064880112835


# Do the same on the Clean Noisy dataset:



In [17]:
testSetNoisy_path = []   # list containing all the fits file path for the test 

for ind in indices:
    key = keys[ind]

    data = os.path.join(data_path, 'clean_noisy', "clean_noisy_gaussians_" + key + ".fits")
    testSetNoisy_path.append(data)

In [18]:
%%capture
listDetectedSourcesNoisy_test = myUtils.detectSourcesFromAllFiles(testSetNoisy_path)

stty: stdin isn't a terminal
stty: stdin isn't a terminal
stty: stdin isn't a terminal
stty: stdin isn't a terminal
stty: stdin isn't a terminal
stty: stdin isn't a terminal
stty: stdin isn't a terminal
stty: stdin isn't a terminal
stty: stdin isn't a terminal
stty: stdin isn't a terminal
stty: stdin isn't a terminal
stty: stdin isn't a terminal
stty: stdin isn't a terminal
stty: stdin isn't a terminal
stty: stdin isn't a terminal
stty: stdin isn't a terminal
stty: stdin isn't a terminal
stty: stdin isn't a terminal
stty: stdin isn't a terminal
stty: stdin isn't a terminal
stty: stdin isn't a terminal
stty: stdin isn't a terminal
stty: stdin isn't a terminal
stty: stdin isn't a terminal
stty: stdin isn't a terminal
stty: stdin isn't a terminal
stty: stdin isn't a terminal
stty: stdin isn't a terminal
stty: stdin isn't a terminal
stty: stdin isn't a terminal
stty: stdin isn't a terminal
stty: stdin isn't a terminal
stty: stdin isn't a terminal
stty: stdin isn't a terminal
stty: stdin is

In [22]:
len(listDetectedSourcesNoisy_test)

916

In [23]:
saveSourceDetection_path = "/Users/patoch/Desktop/Patoch/Uni/2nd-Year/MasterThesis/master-thesis/Dataset.nosync/sourceDetectionNoisy_Karabo"
myUtils.saveSourceDetection(listDetectedSourcesNoisy_test, saveSourceDetection_path, "clean_noisy_gaussians_", indices, keys)

In [24]:
tmp = myUtils.loadSourceDetection(saveSourceDetection_path, "clean_noisy_gaussians_", indices, keys)
print(len(tmp))
print(tmp[0].shape)
print(listDetectedSourcesNoisy_test[0].shape)

916
(0,)
(0,)


In [36]:
resAllComparisonNoisy = myUtils.compareRealAndDetectedSources_pixels(listDetectedSourcesNoisy_test, listRealDetection_test_px)
print("Result (TP, FP, FN): ", resAllComparisonNoisy)

Result (TP, FP, FN):  [ 471   11 2365]


In [37]:
# Compute the TP, FP and FN for each file
listComparisonNoisy_test= []
for ind in tqdm(range(len(listDetectedSourcesNoisy_test))):
    listComparisonNoisy_test.append(myUtils.compareRealAndDetectedSources_pixels([listDetectedSourcesNoisy_test[ind]], [listRealDetection_test_px[ind]]))
listComparisonNoisy_test = np.array(listComparisonNoisy_test)
print("Result (TP, FP, FN): \n", listComparisonNoisy_test)
print()

print("Mean TP: ", np.mean(listComparisonNoisy_test[:,0]))
print("Mean FP: ", np.mean(listComparisonNoisy_test[:,1]))
print("Mean FN: ", np.mean(listComparisonNoisy_test[:,2]))

100%|██████████| 916/916 [00:00<00:00, 14118.34it/s]

Result (TP, FP, FN): 
 [[0 0 2]
 [1 0 2]
 [2 0 3]
 ...
 [0 0 3]
 [0 0 3]
 [0 0 5]]

Mean TP:  0.5141921397379913
Mean FP:  0.012008733624454149
Mean FN:  2.581877729257642


In [40]:
purityNoisyAll = resAllComparisonNoisy[0] / (resAllComparisonNoisy[0] + resAllComparisonNoisy[1])
CompletnessNoisyAll = resAllComparisonNoisy[0] / (resAllComparisonNoisy[0] + resAllComparisonNoisy[2])
print("Purity: ", purityNoisyAll)
print("Completness: ", CompletnessNoisyAll)

Purity:  0.9771784232365145
Completness:  0.1660789844851904


In [38]:
meanNoisyPurity = np.mean(listComparisonNoisy_test[:,0]) / (np.mean(listComparisonNoisy_test[:,0]) + np.mean(listComparisonNoisy_test[:,1]))
meanNoisyCompletness = np.mean(listComparisonNoisy_test[:,0]) / (np.mean(listComparisonNoisy_test[:,0]) + np.mean(listComparisonNoisy_test[:,2]))
print("Mean Purity: ", meanNoisyPurity)
print("Mean Completness: ", meanNoisyCompletness)

Mean Purity:  0.9771784232365144
Mean Completness:  0.1660789844851904
